In [2]:
!pip install spotipy
!pip install pymusixmatch
!pip install textblob

In [15]:
# NLP imports
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# API imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from musixmatch import Musixmatch # lyrics api

# for displaying album covers
from IPython.display import Image, display

# other imports
import json
import requests
import io
import os

Option 1

In [4]:
# nltk downloads needed to run the Natural Language Processing
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\blake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\blake\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\blake\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# function to break down text for NLP purposes
def preprocess(text):
    # define a set of stopwords to be removed from text
    stop_words = set(stopwords.words('english'))
    # a lemmatizer breaks down words into more basic forms
    # ex/ running becomes run
    lemmatizer = WordNetLemmatizer()
    # word_tokenize decomposes a word into individual tokens, which can be used for further analysis
    words = word_tokenize(text.lower())
    # making the text lowercase makes it simpler
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    # breaks down all words in the lyrics as long as they are not designated stop words or are not alphanumeric (in which case they wouldn't be words anyways)
    return " ".join(words)

In [6]:
client_id = input("Please enter your Spotify client ID: ")
client_secret = input("Please enter your Spotify client secret: ")

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
def search_lyrics(query, num):
    # api key needed to use Musixmatch
    musix_key = input("Please enter your personal MusixMatch key: ")
    musixmatch_api = Musixmatch(musix_key)
    base_url = "https://api.musixmatch.com/ws/1.1/"
    endpoint = "track.search"
    # join the base_url and the endpoint to form the full url
    url = f"{base_url}{endpoint}"
    params = {
        "q": query,
        "apikey": musix_key,
        # descending order of track rating
        "s_track_rating": "desc",
        "f_has_lyrics": 1
    }
    # search using the url and the user input
    response = requests.get(url, params=params)
    data = response.json() # put the response in json form
    tracks = []
    count = 0
    for track in data["message"]["body"]["track_list"]:
        # adds song names and the corresponding artist to a list
        if count < num:
          track_name = track['track']['track_name']
          artist_name = track["track"]["artist_name"]
          tracks.append((track_name, artist_name))
          count += 1
        else:
            break
    return tracks

In [23]:
def get_album_cover():
    processed_input = preprocess(input("Enter a lyric: ")) # run the Natural Language Processing
    max_songs = int(input("Enter the maximum number of songs to display (1-10): "))
    res = search_lyrics(processed_input, max_songs)
    for song, artist in res: # unpacks each tuple ex/ (song, artist)
        # use the Spotify search API to search for the song
        try:
            results = sp.search(q='track:' + song + ' artist:' + artist, type='track')
            # get the first result from the search
            track = results['tracks']['items'][0]
            # get the album cover image URL
            image_url = track['album']['images'][0]['url']
            # push image URL and song name+artist
            print(f"{song} by {artist}")
            display(Image(url=image_url))
        except:
            # if the song does not have an image in spotify
            image_url = "Could not load cover image for this song."
        
get_album_cover()

Enter a lyric: it's close to midnight and something evil's lurkin in the dark
Enter the maximum number of songs to display (1-10): 1
Thriller by Michael Jackson
